# Langfuse 기초

### 실습 목표
5일차의 첫 실습에서는, 복잡한 Agent 시스템의 내부를 투명하게 들여다보는 관측 가능성(Observability)을 위하여 `Langfuse`를 활용합니다. 여기에서 시스템을 분석하고, 평가하며, 개선하는 다양한 기능을 살펴보겠습니다.

이번 Part 1에서는 `Langfuse`의 기본적인 작동 원리를 이해하고, 간단한 함수와 LLM 호출을 추적하는 과정을 살펴봅니다.

1. Trace, Span, Observation, Generation 등 `Langfuse` 대시보드를 구성하는 핵심 데이터 모델의 의미와 관계를 이해합니다.
2. `Langfuse`의 가장 간단한 추적 기능인 `@trace()` 데코레이터를 사용하여, 일반 Python 함수의 실행(입력, 출력, 소요 시간, 에러)을 자동으로 추적하는 방법을 학습합니다.
3. `LangChain`으로 구성된 LLM 체인의 실행 과정을 추적하기 위해 `Langfuse`의 `CallbackHandler`를 사용하는 방법을 익히고, LLM 호출과 관련된 정보(토큰 사용량, 비용, 지연 시간 등)가 어떻게 자동으로 기록되는지 확인합니다.

In [ ]:
# !pip install langfuse langchain-google-genai pydantic instructor jsonref -q

In [ ]:
import os

# 1. Google API 키 설정
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

# 2. Langfuse API 키 설정
# https://cloud.langfuse.com/ 에서 생성한 프로젝트의 API 키
os.environ["LANGFUSE_SECRET_KEY"] = "YOUR_API_KEY"
os.environ["LANGFUSE_PUBLIC_KEY"] = "YOUR_API_KEY"
# Host의 경우 클라우드의 국가를 확인하여 설정합니다.
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"

### 1. `@observe()` 데코레이터로 일반 함수 추적하기

`Langfuse`를 사용하는 가장 간단한 방법은 `@observe()` 데코레이터를 추적하고 싶은 함수 위에 붙이는 것입니다. 이 데코레이터는 해당 함수가 호출될 때마다 다음 정보를 자동으로 `Langfuse` 서버로 전송합니다:
- 입력 (Input): 함수에 전달된 모든 인자
- 출력 (Output): 함수가 반환한 결과값
- 성능 (Performance): 함수의 실행 시간 (Latency)
- 에러 (Errors): 함수 실행 중 발생한 예외(Exception) 정보

In [ ]:
from langfuse import Langfuse, observe, get_client

# Langfuse 클라이언트 초기화
# 이 클라이언트는 설정된 환경 변수를 자동으로 읽어 서버와 연결합니다.
langfuse = Langfuse()

langfuse.auth_check()

In [ ]:
@observe()
def simple_calculator(a: int, b: int, operation: str = "add") -> int:
    """두 개의 정수를 받아 간단한 연산을 수행하는 함수"""
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    else:
        raise ValueError("지원하지 않는 연산입니다.")


# 1. 성공 케이스 실행
result_add = simple_calculator(10, 5, operation="add")
print(f"  - 결과: {result_add}")

# 2. 실패 케이스 실행 (에러 추적)
try:
    simple_calculator(10, 5, operation="multiply")
except ValueError as e:
    print(f"  - 예상된 에러 발생: {e}")

#### Langfuse 대시보드 확인 (Action Item)

1.  Langfuse Cloud ([https://cloud.langfuse.com/](https://cloud.langfuse.com/)) 로 이동하여 여러분의 프로젝트에 접속하세요.
2.  왼쪽 메뉴에서 'Traceing'을 클릭합니다.
3.  방금 실행한 `simple_calculator`라는 이름의 새로운 Trace가 두 개 생성된 것을 확인합니다. (성공 케이스 1개, 실패 케이스 1개)
4.  각 Trace를 클릭하여 상세 정보를 확인해보세요:
    - Input/Output: 함수에 전달된 인자와 반환값이 정확히 기록되었는지 확인합니다.
    - Error: 실패한 Trace의 경우, 'Error' 탭에 `ValueError` 정보가 상세히 기록된 것을 확인합니다.
    - Latency: 각 Trace의 실행 시간을 확인합니다.

### 2. LangChain의 '신경망' 추적하기: `CallbackHandler`

Part 1에서 사용한 `@observe()` 데코레이터는 일반 Python 함수를 추적하는 데는 유용하지만, LLM 호출의 풍부한 메타데이터(토큰 사용량, 모델 파라미터, 비용 등)를 자동으로 캡처하지는 못합니다. 이 문제를 해결하기 위해 `Langfuse`는 `LangChain`의 실행 라이프사이클에 직접 연결되는 `CallbackHandler`를 제공합니다.
  
이 핸들러는 LangChain 체인이 실행되는 동안 발생하는 모든 내부 이벤트(`on_llm_start`, `on_llm_end`, `on_chain_start` 등)를 감지하여, LLM 호출과 관련된 상세 정보를 자동으로 `Langfuse`로 전송합니다.

In [ ]:
from langfuse.langchain import CallbackHandler
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. Langfuse 콜백 핸들러 초기화
# 핸들러를 초기화할 때, session_id나 user_id와 같은 메타데이터를 전달하여
# Trace를 그룹화하고 필터링하는 데 사용할 수 있습니다.
langfuse_handler = CallbackHandler()
langfuse_handler.session_id = "my-first-llm-session"
langfuse_handler.user_id = "samsung-dx-researcher"

In [ ]:
# 2. LangChain 구성 요소 초기화
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 사용자의 질문에 대해 창의적이고 상세한 답변을 제공하는 AI 어시스턴트입니다."),
        ("user", "{input}"),
    ]
)
output_parser = StrOutputParser()

# 3. LCEL(LangChain Expression Language)을 사용한 체인 구성
chain = prompt | llm | output_parser

#### LangChain 체인 실행 및 자동 추적

이제 구성된 체인을 실행합니다. 여기서 핵심은 `.invoke()` 메소드를 호출할 때 `config` 딕셔너리에 우리가 만든 `langfuse_handler`를 `callbacks` 리스트에 담아 전달하는 것입니다. 이 간단한 작업만으로, LangChain은 체인 실행의 모든 단계를 Langfuse 핸들러에게 자동으로 알려주고, 핸들러는 이 정보를 `Langfuse` 서버로 전송합니다.

In [ ]:
@observe()
def run_langchain_with_trace():
    print("--- LangChain 체인 실행 및 Langfuse Tracing 시작 ---")

    user_question = "인공지능 에이전트의 미래에 대해 가장 흥미로운 응용 분야 3가지를 알려줘."

    # CallbackHandler 초기화
    langfuse_handler = CallbackHandler()

    # config에 콜백 핸들러를 전달하여 체인 실행
    response = chain.invoke({"input": user_question}, config={"callbacks": [langfuse_handler]})

    print("\n--- LLM 응답 ---")
    print(response)

    # @observe() 데코레이터가 있어야 current trace URL을 가져올 수 있음
    langfuse = get_client()
    trace_url = langfuse.get_trace_url()

    print("\n" + "-" * 50)
    print(f"체인 실행 완료 - 아래 URL에서 상세한 Trace를 확인해보세요.")
    print(trace_url)

    return response


# 함수 실행
run_langchain_with_trace()

#### Langfuse 대시보드 심층 분석 (Action Item)

1. 위에서 출력된 Trace URL을 클릭하여 `Langfuse` 대시보드로 이동하세요.
2. 이번 Trace는 Part 1의 것과 어떻게 다른지 유심히 확인해봅시다.
    - Trace가 더 이상 단일 `Span`이 아닙니다. 전체 체인 실행(`langchain-google-genai-run`)을 나타내는 부모(Parent) Trace와, 그 안에서 실제 LLM 호출이 일어난 자식(Child) Generation으로 구성된 계층 구조를 확인할 수 있습니다.
    - `ChatGoogleGenerativeAI`을 클릭하여 상세 정보를 확인하세요.
        - Model: `gemini-2.5-pro`와 같이 사용된 모델 이름이 자동으로 기록됩니다.
        - Prompt & Completion: LLM에 전달된 전체 프롬프트와 LLM이 생성한 최종 결과물이 구분되어 표시됩니다.
        - Usage & Cost: 입력/출력 토큰 수와, Langfuse가 자동으로 계산한 예상 비용이 표시됩니다. 이를 통해 우리는 Agent의 운영 비용을 정량적으로 추적할 수 있습니다.
    - 세션 및 사용자 정보: Trace 상세 정보의 상단에서, 우리가 핸들러에 설정했던 `Session ID`와 `User ID`가 올바르게 태깅(Tagging)되었는지 확인하세요. 이를 통해 나중에 특정 사용자나 세션의 모든 활동을 필터링하여 볼 수 있습니다.

---

### 3. 수동 제어를 통한 심층 추적: `Spans`와 `Metadata`

자동 추적은 편리하지만, 때로는 LLM 호출 외의 중요한 비즈니스 로직(예: 데이터 전처리, Tool 실행, 결과 후처리)의 성능과 내용을 추적하고 싶을 때가 있습니다. `Langfuse`는 개발자가 직접 Trace 내부에 'Span'이라는 작업 단위를 생성하여, 워크플로우의 모든 단계를 수동으로, 그리고 계층적으로 추적할 수 있는 기능도 제공합니다.

- `langfuse.trace()`: 컨텍스트 관리자 Trace 객체를 생성하고 수동으로 span을 추가하여 Trace의 생성과 종료를 명시적으로 제어합니다.
- `trace.span()`: Trace 내부에 중첩된 작업 단위(Span)를 생성하여, 복잡한 워크플로우를 논리적인 단계로 나누어 추적합니다.
- `metadata` 와 `tags`: Trace와 Span에 커스텀 메타데이터(e.g., `prompt_version: 'v2'`)와 태그(e.g., `'rag'`, `'summarization'`)를 추가하여, 나중에 수많은 Trace를 쉽게 검색하고 필터링하는 방법을 배웁니다.

In [ ]:
import time
from langfuse import get_client
from langfuse.langchain import CallbackHandler

# Langfuse 클라이언트 가져오기
langfuse = get_client()

In [ ]:
def get_rag_answer(query: str):
    """간단한 RAG 파이프라인을 시뮬레이션하고, 각 단계를 수동으로 추적하는 함수"""

    # 1. 최상위 Trace 생성: start_as_current_span 컨텍스트 매니저 사용
    with langfuse.start_as_current_span(name="rag_pipeline", input={"query": query}) as root_span:
        # 트레이스 메타데이터 설정
        root_span.update_trace(
            user_id="samsung-dx-researcher",
            session_id="rag-session-001",
            metadata={"prompt_version": "v1.2", "retriever_type": "basic"},
            tags=["rag", "production"],
        )

        # 2. 첫 번째 Span: 쿼리 전처리
        print("--- [Step 1] 쿼리 전처리 중... ---")
        with langfuse.start_as_current_span(name="step_1_preprocess_query", input={"query": query}) as span_1:
            time.sleep(0.5)
            processed_query = query.lower().strip() + " AI 에이전트"
            span_1.update(output={"processed_query": processed_query})

        # 3. 두 번째 Span: 외부 데이터베이스 검색 (벡터 데이터베이스에서 문서가 회수되는 과정을 시뮬레이션)
        print("--- [Step 2] 관련 문서 검색 중... ---")
        with langfuse.start_as_current_span(
            name="step_2_retrieve_documents", input={"query": processed_query}
        ) as span_2:
            time.sleep(1)
            retrieved_docs = [
                "LangGraph는 동적인 제어 흐름을 만드는 데 강점이 있습니다.",
                "CrewAI는 빠르게 Multi-Agent 팀을 구성하는 데 특화되어 있습니다.",
            ]
            span_2.update(output={"documents_found": len(retrieved_docs)})

        # 4. 세 번째 Span: LLM을 사용한 최종 답변 생성 (Generation)
        print("--- [Step 3] LLM으로 최종 답변 생성 중... ---")
        with langfuse.start_as_current_generation(
            name="step_3_generate_answer", model="gemini-2.5-flash"
        ) as generation:
            context = "\n".join(retrieved_docs)
            prompt = f"다음 정보를 바탕으로 '{query}'에 대해 한 문장으로 답변해줘: {context}"

            # LLM 호출 (이전 파트의 llm 객체 사용)
            response = llm.invoke(prompt)
            final_answer = response.content

            # Generation 업데이트
            generation.update(
                input=prompt, output=final_answer, usage=response.response_metadata.get("token_usage", {})
            )

        # 최종 결과를 root span에 기록
        root_span.update(output={"final_answer": final_answer})

        return final_answer

In [ ]:
final_rag_answer = get_rag_answer("LangGraph와 CrewAI의 차이점")
print(f"\n--- 최종 답변 ---\n{final_rag_answer}")

print(f"\n실행 완료")

# Langfuse 대시보드에서 계층적으로 구성된 'rag_pipeline' Trace를 확인해보세요.

# 짧은 생명주기 애플리케이션에서는 flush 호출 필요
langfuse.flush()

### 4. 결과 평가 및 품질 관리: `Scores`

Agent의 실행 과정을 추적하는 것만으로는 충분하지 않습니다. 우리는 그 결과물이 얼마나 '좋은지'를 정량적으로 평가하고, 이 평가 기록을 추적 데이터와 함께 저장해야 합니다. `Langfuse`의 `Score` 기능은 바로 이 문제를 해결합니다.

#### 4.1. LLM-as-a-Judge: LLM을 이용한 자동 평가

수천, 수만 건의 Agent 응답을 사람이 일일이 평가하는 것은 불가능합니다. LLM-as-a-Judge는 이 문제를 해결하는 기법으로, 평가자 역할을 하는 또 다른 LLM을 사용하여 Agent의 응답 품질을 자동으로 채점하는 방식입니다.

`instructor`와 `Pydantic`을 활용하여, 평가자 LLM이 일관된 형식으로 평가 점수와 근거를 반환하도록 만들고, 그 결과를 `Langfuse`의 `Score`로 기록하는 자동화된 평가 파이프라인을 구축합니다.

In [ ]:
from langfuse import get_client, observe
import instructor
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langfuse.langchain import CallbackHandler
import time
import os

# Langfuse 클라이언트 및 LLM 초기화
langfuse = get_client()

# 최신 instructor API 사용 - from_provider 방식
evaluator_llm_client = instructor.from_provider(
    "google/gemini-2.5-flash-lite", api_key=os.environ.get("GOOGLE_API_KEY")
)

In [ ]:
# --- 1. 평가 결과를 담을 Pydantic 모델 ---
class EvaluationResult(BaseModel):
    score: float = Field(description="평가 기준에 따른 점수 (0.0 ~ 1.0)")
    reasoning: str = Field(description="점수를 매긴 이유에 대한 상세한 설명")

In [ ]:
# --- 2. LLM-as-a-Judge 함수 ---
def evaluate_answer_with_llm(question: str, answer: str) -> EvaluationResult:
    """LLM을 사용하여 Agent의 답변을 평가하고, 점수와 근거를 반환합니다."""
    prompt = f"""당신은 엄격한 품질 평가자입니다.
    다음 질문과 답변을 보고, 답변이 얼마나 질문의 의도를 잘 파악하고, 명확하며, 유용한 정보를 담고 있는지 평가해주세요.
    점수는 0.0(매우 나쁨)부터 1.0(매우 좋음)까지 매겨주세요.

    질문: {question}
    답변: {answer}
    """

    evaluation = evaluator_llm_client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}], response_model=EvaluationResult
    )
    return evaluation

In [ ]:
# --- 3. 평가 파이프라인 실행 ---
@observe()
def run_and_evaluate_agent():
    print("--- LangChain 체인 실행 및 자동 평가 시작 ---")

    # (1) 에이전트 실행 (이전 파트의 LangChain 체인 재사용)
    handler = CallbackHandler()

    user_question = "AI 에이전트 기술의 가장 큰 도전 과제는 무엇인가요?"
    agent_answer = chain.invoke({"input": user_question}, config={"callbacks": [handler]})

    print(f"\n--- Agent 답변 ---\n{agent_answer}")

    # (2) LLM-as-a-Judge를 통한 자동 평가
    print("\n--- 🤖 LLM으로 자동 평가 중... ---")
    evaluation = evaluate_answer_with_llm(user_question, agent_answer)
    print(f"  - 점수: {evaluation.score}")
    print(f"  - 평가 이유: {evaluation.reasoning}")

    # (3) 평가 결과를 Langfuse Score로 기록
    # @observe() 데코레이터가 있어야 current trace를 사용할 수 있음
    langfuse = get_client()

    # 현재 trace에 score 추가
    langfuse.score_current_trace(name="llm-judge-clarity", value=evaluation.score, comment=evaluation.reasoning)

    # trace URL 가져오기
    trace_url = langfuse.get_trace_url()

    print(f"\n✅ 평가 결과가 Langfuse에 Score로 기록되었습니다.")
    print(f"   Trace URL: {trace_url}")

    return agent_answer


# 함수 실행
run_and_evaluate_agent()

#### Langfuse 대시보드 확인 (Action item)

1.  위에서 출력된 Trace URL을 클릭하여 `Langfuse` 대시보드로 이동하세요.
2.  Trace 상세 페이지의 탭에서 'Scores' 섹션을 확인하세요.
3.  `llm-judge-clarity`라는 이름으로, 우리가 LLM을 통해 자동으로 매긴 점수와 평가 이유가 기록된 것을 볼 수 있습니다.
4.  (심화) 왼쪽 메뉴의 'Scores' 탭으로 이동하면, 프로젝트의 모든 Trace에 대해 매겨진 점수들을 모아서 보거나, 특정 평가 지표(`name`)에 대한 평균 점수나 점수 분포를 분석할 수도 있습니다. 